In [1]:
import os, sys
import pandas as pd
import numpy as np
import re

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
sys.path.append('../src')

from fixtures import df_to_json_list, write_fixture_to_json

In [2]:
def load_sheet_from_xl(fname, sheet_name):
    """
    load sheet from xl file
    """
    fldr_path = '..','data','raw'
    fpath = os.path.join(*fldr_path, fname)
    xl = pd.ExcelFile(fpath)
    df = xl.parse(sheet_name)
        
    return df

In [3]:
def clean_df_resources_languages(df):
    
    cols_to_keep = ['index', 'language','index_language', 'equivalentClasses',]   # language hyperlinks example url https://bioportal.bioontology.org/ontologies/SNOMEDCT/?p=classes&conceptid=http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FSNOMEDCT%2F297301005
    df = df.loc[: ,cols_to_keep]

    #handle NaNs
    df['index_language'] = df['index_language'].astype(pd.Int64Dtype()) # allows column to have NaNs pd.Int64Dtype()
    df['equivalentClasses'] = df['equivalentClasses'].astype(str)

    df = df.set_index('index')
    
    #rename cols # to avoid a naming clashes in django.
    rename_dict = {'language':'name', 
                   'index_language':'parent_language'}
    
    df = df.rename(columns=rename_dict)

    return df

In [4]:
def clean_df_disorders_disorder_categories(df):
    
    cols_to_keep = ['index', 'disorder_category']   # language hyperlinks example url https://bioportal.bioontology.org/ontologies/SNOMEDCT/?p=classes&conceptid=http%3A%2F%2Fpurl.bioontology.org%2Fontology%2FSNOMEDCT%2F297301005
    df = df.loc[: ,cols_to_keep]

    #handle NaNs
    df = df.dropna()
    df = df.set_index('index')
    
    #rename cols # to avoid a naming clashes in django.
    rename_dict = {'disorder_category':'name'}
    df = df.rename(columns=rename_dict)

    return df
    

In [5]:
def create_authors_df():
    """
    the authors column in the questionnaires__assessments sheet has been entered as free text.
    This function creates a dataframe of individual authors to be used to popoulate an authors table
    """    
    sheet_name = 'questionnaires'
    fname = 'assessments.xlsx'
    
    df = load_sheet_from_xl(fname, sheet_name)
    
    
    

In [6]:
def get_surname_and_initials_from_string(in_string):
    """use regex to extract inidividual author names & initials from list of authors
    returns: list of tuples, [(surname, first_initial, middle_initial),]
    
    example: in_string='Achenbach, T. M., & Rescorla, L. A.'
            returns [('Achenbach','T','M'), ('Rescorla','L','A')]
    
    """
    
    expression = "([A-Z][a-z]+?[A-Za-z\s]+)([,]?)([\s]?)([A-Za-z]?)([.]?[,\s]?)([A-Z]?[,]?)"
    matches = re.findall(expression, in_string)

    names_lst = []
    for match in matches:
        surname = match[0]
        first_initial =  match[3]
        middle_initial = match[-1]
        
        names_lst.append((surname, first_initial, middle_initial ))
    
    return names_lst

In [7]:

def make_names_df_from_author_lists(ser_authors):
    """
    ser_authors:pd.Series, with lists of authors as a string
    returns: pd.DataFrame with column 'surname','first_initial','middle_initial'
    """

    ser_authors = ser_authors.drop_duplicates().dropna()
    ser_authors = ser_authors.astype(str)

    overall_authors_lst = []

    for auth_lst in ser_authors:
        # add each set of match results to the overall list
        overall_authors_lst += get_surname_and_initials_from_string(auth_lst)

    df = pd.DataFrame(overall_authors_lst, columns = ['surname','first_initial','middle_initial',] )
    df = df.drop_duplicates()
    
    return df


In [8]:
def create_authors_fixture():
    """
    special case function to create an authors fixture
    """
    sheet_name = 'questionnaires'
    input_fname = 'assessments.xlsx'
    app_name = 'assessments'
    model_name = 'Author'
    output_fname = 'Author.json'
    df = load_sheet_from_xl(input_fname, sheet_name)

    ser_authors = df['authors']
    df = make_names_df_from_author_lists(ser_authors)
    df = df.replace('',np.nan)
    
    fname, fixture_lst = df_to_json_list(df,
                                        app_name,
                                        model_name,
                                        file_name_modifier='',
                                        use_df_index_as_pk=False,
                                        create_datetimefield_name=None,
                                        created_by_field_name=None)
    write_fixture_to_json(fixture_lst, output_fname, output_folder='default')
    
    return

In [46]:
def clean_df_disorders_severities(df):

    df = df.set_index('index')
    keep_cols = ['severity', 'definition']
    df = df.loc[:,keep_cols]

    return df

In [79]:
def clean_df_disorders_disorders(df):

    keep_cols = ['S',
                 'index_disorder_category',
                #  'index_disorder_subcategory',
                #  'index_disorder_subsubcategory',
                #  'index_disorder_subsubsubcategory',
                 'disorder',
                #  'equivalentClasses',
                 'ICD9CM',
                 'ICD10CM',
                 'index_diagnostic_specifier',
                #  'index_diagnostic_inclusion_criterion',
                #  'index_diagnostic_inclusion_criterion2',
                #  'index_diagnostic_exclusion_criterion',
                #  'index_diagnostic_exclusion_criterion2',
                 'index_severity',
                 'note']

    df = df.loc[:, keep_cols]
    df.columns = [col.lower() for col in df.columns]
    df['s'] = df['s'].astype(int)
    df = df.rename(columns={'s':'index'})
    df = df.set_index('index')

    df['index_severity'] = df['index_severity'].astype(pd.Int64Dtype())

    return df

In [104]:
def clean_df_disorders_diagnostic_specifiers(df):

    df = df.dropna(how='all')
    df = df.set_index('index')
    df['equivalentClasses'] = df['equivalentClasses'].astype(str)
    
    return df

In [103]:
def cleaning_func_factory(fname, sheet_name):
    """
    return the appropriate df cleaning function by looking up a dictionary
    the functions should be saved in the dictionary in the format FileName__SheetName
    """
    file_sheet = fname.split('.')[0] + "__" + sheet_name
    
    cleaning_funcs = {}
    cleaning_funcs["resources__languages"] = clean_df_resources_languages
    cleaning_funcs["disorders__disorder_categories"] = clean_df_disorders_disorder_categories
    cleaning_funcs['disorders__severities'] = clean_df_disorders_severities
    cleaning_funcs['disorders__disorders'] = clean_df_disorders_disorders
    cleaning_funcs['disorders__diagnostic_specifiers'] = clean_df_disorders_diagnostic_specifiers
#     cleaning_funcs[''] = 
    
    return cleaning_funcs[file_sheet]

In [72]:
def process_one_sheet(fname, sheet_name, app_name, model_name):
    
    df = load_sheet_from_xl(fname, sheet_name)
    # look up the cleaning function based on file and sheet name 
    df = cleaning_func_factory(fname, sheet_name)(df)

    fname, fixture_lst = df_to_json_list(df,
                                        app_name,
                                        model_name,
                                        file_name_modifier='',
                                        use_df_index_as_pk=True,
                                        create_datetimefield_name=None,
                                        created_by_field_name=None)

    write_fixture_to_json(fixture_lst, fname, output_folder='default')
    
    return

In [105]:
inputs = {
    'sheet_name' : 'diagnostic_specifiers',
    'fname' : 'disorders.xlsx',
    'model_name' : 'DiagnosticSpecifier',
    'app_name' : 'disorders',
    }

sheet_name = inputs['sheet_name']
fname = inputs['fname']
model_name = inputs['model_name']
app_name = inputs['app_name']

In [106]:
df = load_sheet_from_xl(fname, sheet_name)


In [107]:
df = clean_df_disorders_diagnostic_specifiers(df)

# MAIN

In [109]:
# uncomment the dictionaries to run all sheets

inputs_lst = [
#     {
#     'sheet_name' : 'languages',
#     'fname' : 'resources.xlsx',
#     'model_name' : 'Language',
#     'app_name' : 'resources',
#     },
#     {
#     'sheet_name' : 'disorder_categories',
#     'fname' : 'disorders.xlsx',
#     'model_name' : 'DisorderCategory',
#     'app_name' : 'disorders',
#     },
#     {
#     'sheet_name' : 'severities',
#     'fname' : 'disorders.xlsx',
#     'model_name' : 'Severity',
#     'app_name' : 'disorders',
#     },
#     {
#     'sheet_name' : 'disorders',
#     'fname' : 'disorders.xlsx',
#     'model_name' : 'Disorder',
#     'app_name' : 'disorders',
#     },
    {
    'sheet_name' : 'diagnostic_specifiers',
    'fname' : 'disorders.xlsx',
    'model_name' : 'DiagnosticSpecifier',
    'app_name' : 'disorders',
    }
    ]

In [110]:
for d in inputs_lst:
    try: 
        process_one_sheet(d['fname'], d['sheet_name'], d['app_name'], d['model_name'])
    except FileExistsError as e:
        print(e)

wrote fixture to ../data/processed/fixtures/DiagnosticSpecifier.json


In [12]:
create_authors_fixture()

wrote fixture to ../data/processed/fixtures/Author.json
